In [11]:
from pathlib import Path
import numpy as np
import torch
from typing import List
from torch.nn.utils.rnn import pad_sequence
from mltrainer import rnn_models, Trainer
from torch import optim

from mads_datasets import datatools
import mltrainer
mltrainer.__version__

'0.2.5'

In [12]:
from mads_datasets import DatasetFactoryProvider, DatasetType
from mltrainer.preprocessors import PaddedPreprocessor
preprocessor = PaddedPreprocessor()

gesturesdatasetfactory = DatasetFactoryProvider.create_factory(DatasetType.GESTURES)
streamers = gesturesdatasetfactory.create_datastreamer(batchsize=32, preprocessor=preprocessor)
train = streamers["train"]
valid = streamers["valid"]

2025-10-04 22:40:52.761 | INFO     | mads_datasets.base:download_data:121 - Folder already exists at C:\Users\mwien\.cache\mads_datasets\gestures
100%|██████████| 651/651 [00:00<00:00, 3039.54it/s]


In [13]:
trainstreamer = train.stream()
validstreamer = valid.stream()
x, y = next(iter(trainstreamer))

In [14]:
from mltrainer import TrainerSettings, ReportTypes
from mltrainer.metrics import Accuracy

accuracy = Accuracy()


In [15]:
loss_fn = torch.nn.CrossEntropyLoss()

In [16]:
import torch
if torch.backends.mps.is_available() and torch.backends.mps.is_built():
    device = torch.device("mps")
    print("Using MPS")
elif torch.cuda.is_available():
    device = "cuda:0"
    print("using cuda")
else:
    device = "cpu"
    print("using cpu")

# on my mac, at least for the BaseRNN model, mps does not speed up training
# probably because the overhead of copying the data to the GPU is too high
# so i override the device to cpu
device = "cpu"
# however, it might speed up training for larger models, with more parameters

using cpu


Set up the settings for the trainer and the different types of logging you want

In [17]:
settings = TrainerSettings(
    epochs=16, # increase this to about 100 for training
    metrics=[accuracy],
    logdir=Path("gestures"),
    train_steps=len(train),
    valid_steps=len(valid),
    reporttypes=[ReportTypes.TOML, ReportTypes.TENSORBOARD, ReportTypes.MLFLOW],
    scheduler_kwargs={"factor": 0.5, "patience": 5},
    earlystop_kwargs = {
        "save": True, # save every best model, and restore the best one
        "verbose": True,
        "patience": 5, # number of epochs with no improvement after which training will be stopped
        "delta": 0.0, # minimum change to be considered an improvement
    }
)
settings

epochs: 16
metrics: [Accuracy]
logdir: gestures
train_steps: 81
valid_steps: 20
reporttypes: [<ReportTypes.TOML: 'TOML'>, <ReportTypes.TENSORBOARD: 'TENSORBOARD'>, <ReportTypes.MLFLOW: 'MLFLOW'>]
optimizer_kwargs: {'lr': 0.001, 'weight_decay': 1e-05}
scheduler_kwargs: {'factor': 0.5, 'patience': 5}
earlystop_kwargs: {'save': True, 'verbose': True, 'patience': 5, 'delta': 0.0}

In [ ]:
import torch.nn as nn
import torch
from torch import Tensor
from dataclasses import dataclass

@dataclass
class ModelConfig:
    input_size: int
    hidden_size: int
    num_layers: int
    output_size: int
    dropout: float = 0.0

class LSTMmodel(nn.Module):
    def __init__(
        self,
        config,
    ) -> None:
        super().__init__()
        self.config = config
        self.lstm = nn.LSTM(
            input_size=config.input_size,
            hidden_size=config.hidden_size,
            dropout=config.dropout,
            batch_first=True,
            num_layers=config.num_layers,
        )
        self.linear = nn.Linear(config.hidden_size, config.output_size)

    def forward(self, x: Tensor) -> Tensor:
        x, _ = self.lstm(x)
        last_step = x[:, -1, :]
        yhat = self.linear(last_step)
        return yhat

In [ ]:
import mlflow
from datetime import datetime

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("gestures")
modeldir = Path("gestures").resolve()
if not modeldir.exists():
    modeldir.mkdir(parents=True)
    
n_repeats = 3

for repeat in range(n_repeats):
    print(f"Repeat {repeat+1}/{n_repeats}")
    
    with mlflow.start_run():
        # Set MLflow tags to record metadata about the model and developer
        mlflow.set_tag("model", f"{repeat}_GRU_16epochs_0.5drop_128hidsize_LSTM")
        mlflow.set_tag("dev", "Marcello")
        # Log hyperparameters to MLflow

        mlflow.log_param("epochs", settings.epochs)

        mlflow.log_param("learning_rate", settings.optimizer_kwargs.get("lr", None))
        
        config = ModelConfig(
            input_size=3,
            hidden_size=128,
            num_layers=1,
            output_size=20,
            dropout=0.5,
        )

        model = LSTMmodel(
            config=config,
        )

        trainer = Trainer(
            model=model,
            settings=settings,
            loss_fn=loss_fn,
            optimizer=optim.Adam,
            traindataloader=trainstreamer,
            validdataloader=validstreamer,
            scheduler=optim.lr_scheduler.ReduceLROnPlateau,
            device=device,
        )
        trainer.loop()

        if not settings.earlystop_kwargs["save"]:
            tag = datetime.now().strftime("%Y%m%d-%H%M-")
            modelpath = modeldir / (tag + "model.pt")
            torch.save(model, modelpath)

c:\Master Applied Data Science\Year 2\Semester 3 (Deep Learning & Model Deployment)\Portfolio-Marcello-Wienhoven\.venv\Lib\site-packages\torch\nn\modules\rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn(
2025-10-04 22:40:54.378 | INFO     | mltrainer.trainer:dir_add_timestamp:24 - Logging to gestures\20251004-224054
2025-10-04 22:40:54.380 | INFO     | mltrainer.trainer:__init__:68 - Found earlystop_kwargs in settings.Set to None if you dont want earlystopping.


Repeat 1/3


100%|██████████| 81/81 [00:01<00:00, 55.54it/s]
2025-10-04 22:40:56.032 | INFO     | mltrainer.trainer:report:209 - Epoch 0 train 2.7006 test 2.3617 metric ['0.1688']
2025-10-04 22:40:56.033 | INFO     | mltrainer.trainer:save_checkpoint:268 - Validation loss (2.3617 --> 2.3617).Saving gestures\20251004-224054\checkpoint.pt ...
100%|██████████| 81/81 [00:01<00:00, 53.89it/s]
2025-10-04 22:40:57.698 | INFO     | mltrainer.trainer:report:209 - Epoch 1 train 2.1623 test 2.0010 metric ['0.3047']
2025-10-04 22:40:57.698 | INFO     | mltrainer.trainer:save_checkpoint:268 - Validation loss (2.3617 --> 2.0010).Saving gestures\20251004-224054\checkpoint.pt ...
100%|██████████| 81/81 [00:01<00:00, 59.52it/s]
2025-10-04 22:40:59.221 | INFO     | mltrainer.trainer:report:209 - Epoch 2 train 1.8016 test 1.6109 metric ['0.3984']
2025-10-04 22:40:59.222 | INFO     | mltrainer.trainer:save_checkpoint:268 - Validation loss (2.0010 --> 1.6109).Saving gestures\20251004-224054\checkpoint.pt ...
100%|█████

Repeat 2/3


100%|██████████| 81/81 [00:01<00:00, 58.64it/s]
2025-10-04 22:41:26.189 | INFO     | mltrainer.trainer:report:209 - Epoch 0 train 2.6973 test 2.3499 metric ['0.1969']
2025-10-04 22:41:26.189 | INFO     | mltrainer.trainer:save_checkpoint:268 - Validation loss (2.3499 --> 2.3499).Saving gestures\20251004-224124\checkpoint.pt ...
100%|██████████| 81/81 [00:01<00:00, 58.48it/s]
2025-10-04 22:41:27.735 | INFO     | mltrainer.trainer:report:209 - Epoch 1 train 2.1497 test 1.9970 metric ['0.2922']
2025-10-04 22:41:27.736 | INFO     | mltrainer.trainer:save_checkpoint:268 - Validation loss (2.3499 --> 1.9970).Saving gestures\20251004-224124\checkpoint.pt ...
100%|██████████| 81/81 [00:01<00:00, 60.27it/s]
2025-10-04 22:41:29.238 | INFO     | mltrainer.trainer:report:209 - Epoch 2 train 1.7766 test 1.6715 metric ['0.4031']
2025-10-04 22:41:29.239 | INFO     | mltrainer.trainer:save_checkpoint:268 - Validation loss (1.9970 --> 1.6715).Saving gestures\20251004-224124\checkpoint.pt ...
100%|█████

Repeat 3/3


100%|██████████| 81/81 [00:01<00:00, 44.63it/s]
2025-10-04 22:41:57.201 | INFO     | mltrainer.trainer:report:209 - Epoch 0 train 2.6529 test 2.3315 metric ['0.1484']
2025-10-04 22:41:57.202 | INFO     | mltrainer.trainer:save_checkpoint:268 - Validation loss (2.3315 --> 2.3315).Saving gestures\20251004-224155\checkpoint.pt ...
100%|██████████| 81/81 [00:01<00:00, 41.68it/s]
2025-10-04 22:41:59.338 | INFO     | mltrainer.trainer:report:209 - Epoch 1 train 2.1553 test 1.9526 metric ['0.3109']
2025-10-04 22:41:59.339 | INFO     | mltrainer.trainer:save_checkpoint:268 - Validation loss (2.3315 --> 1.9526).Saving gestures\20251004-224155\checkpoint.pt ...
100%|██████████| 81/81 [00:01<00:00, 43.45it/s]
2025-10-04 22:42:01.405 | INFO     | mltrainer.trainer:report:209 - Epoch 2 train 1.8188 test 1.6592 metric ['0.4641']
2025-10-04 22:42:01.406 | INFO     | mltrainer.trainer:save_checkpoint:268 - Validation loss (1.9526 --> 1.6592).Saving gestures\20251004-224155\checkpoint.pt ...
100%|█████

In [20]:
mlflow.end_run()

## Open MLFlow
"""
```bash
mlflow server \
    --backend-store-uri sqlite:///mlflow.db \
    --host 127.0.0.1 \ 
    --port 5000 \
        
mlflow server --backend-store-uri sqlite:///3-hypertuning-rnn/mlflow.db --host 127.0.0.1 --port 5000
```
"""